In [5]:
import os
import nengo_dl
import numpy as np
import keras
import tensorflow as tf
import sklearn
from keras import Sequential
from tensorflow.python.keras import Model
from tensorflow.python.keras.callbacks import EarlyStopping
from tensorflow.python.keras.layers import Conv2D, BatchNormalization, Dropout, AveragePooling2D, Flatten, Dense

dataset_path = os.path.join('..', 'datasets', 'output', 'p300-target-nontarget.npz')
dataset = np.load(dataset_path)

target_x, non_target_x = dataset['target_features'], dataset['non_target_features']

In [6]:
features = np.concatenate((target_x, non_target_x), axis=0)
target_labels = np.tile(np.array([1, 0]), (target_x.shape[0], 1))
non_target_labels = np.tile(np.array([0, 1]), (non_target_x.shape[0], 1))

labels = np.vstack((target_labels, non_target_labels))
features = np.expand_dims(features, -1)

In [ ]:
train_x, train_y, test_x, test_y = sklearn.model_selection.train_test_split(
    features, labels, test_size=0.25, shuffle=True, random_state=0
)
save_callback = EarlyStopping(patience=5, verbose=1, restore_best_weights=True)
shuffle_split = sklearn.model_selection.ShuffleSplit(n_splits=30, test_size=0.25, random_state=0)

In [ ]:
def create_model():
    conv_input = Conv2D(filters=6, kernel_size=(3, 3))
    batch_norm1 = BatchNormalization()(conv_input)
    dropout1 = Dropout(0.5)(batch_norm1)
    avg_pooling = AveragePooling2D(pool_size=(1, 8))(dropout1)
    flatten = Flatten()(avg_pooling)
    dense1 = Dense(100, activation=keras.activations.elu)(flatten)
    batch_norm2 = BatchNormalization()(dense1)
    dropout2 = Dropout(0.5)(batch_norm2)
    dense_output = Dense(2, activation='softmax')(dropout2)

    return Model(inputs= conv_input, outputs=dense_output), conv_input, dense_output

In [ ]:

def validate(validation_metrics, test_metrics, snn_metrics):
    model, input_layer, output_layer = create_model()
    converter = nengo_dl.Converter(model)

    with nengo_dl.Simulator(converter.net, minibatch_size=10) as sim:
        sim.compile(
            optimizer=keras.optimizers.Adam(),
            loss=keras.losses.BinaryCrossentropy(),
            metrics=[keras.metrics.binary_accuracy]
        )

        validation_metrics.append(
            sim.fit(
                callbacks=[save_callback],
                shuffle=True,
                epochs=30,
                batch_size=16,
                validation_data=(
                    { converter.inputs[input_layer]: train_x[validation] },
                    { converter.outputs[output_layer]: train_y[validation] }
                )
            )
        )

        acc, loss = sim.evaluate(
            { converter.inputs[input_layer]: test_x },
            { converter.outputs[output_layer]: test_y }
        )
        test_metrics.append({'accuracy': acc, 'loss': loss})

        sim.save_params('./keras_to_snn_params')



validation_metrics, test_metrics, snn_metrics = [], [], []

i = 0
for train, validation in shuffle_split(train_x):
    print('Iteration {}'.format(i))
    validate(validation_metrics, test_metrics, snn_metrics)
